In [ ]:
# インポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import psutil 

from sklearn.preprocessing import LabelEncoder

In [ ]:
# cd C:\Users\minih\python_prac1\library\m5

C:\Users\minih\python_prac1\library\m5


In [ ]:
datas = pd.read_csv("sales_train_validation.csv")
datas

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [ ]:
# @auth Andrew Lukyanenko
# メモリサイズの削減
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# Label-Encoding
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)

    return df

datas = encode_categorical(
   datas, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

Mem. usage decreased to 94.01 Mb (78.9% reduction)


In [ ]:
# 一意なidを取得
product = datas[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

In [ ]:
product.to_pickle("product.pickle")

In [ ]:
product

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0
...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,1432,2,0,9,2
30486,FOODS_3_824_WI_3_validation,1433,2,0,9,2
30487,FOODS_3_825_WI_3_validation,1434,2,0,9,2
30488,FOODS_3_826_WI_3_validation,1435,2,0,9,2


In [1]:
N = 2
START_COLUMN = 6
END_COLUMN = START_COLUMN + 1913 - (365 * N)

# 最新n年分を残す
datas.drop(datas.columns[START_COLUMN:END_COLUMN],axis=1,inplace=True)

NameError: ignored

In [ ]:
datas.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_819,d_820,d_821,d_822,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
test = pd.read_csv("sample_submission.csv")

In [ ]:
test.head(2)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# F1をd_1914のように対応付け
keys = []
values = []
for i in np.arange(28):
    keys.append(test.columns[1+i])
    values.append("d_" + str(1914+i))
maps = dict(zip(keys,values))

test.rename(columns=maps,inplace=True)

In [ ]:
test.head(2)

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# NaN値の設定
test.loc[:,test.columns[1:]] = np.NaN

In [ ]:
# 訓練とテストをいったんくっつける
datas = pd.merge(datas,test)

In [ ]:
datas

,id,item_id,dept_id,cat_id,store_id,state_id,d_1184,d_1185,d_1186,d_1187,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,1432,2,0,9,2,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30486,FOODS_3_824_WI_3_validation,1433,2,0,9,2,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30487,FOODS_3_825_WI_3_validation,1434,2,0,9,2,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30488,FOODS_3_826_WI_3_validation,1435,2,0,9,2,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 整然データに変換
datas = pd.melt(datas ,id_vars=datas.columns[0:6], var_name='date', value_name='sell_num')

In [ ]:
datas.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,date,sell_num
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1184,0.0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1184,0.0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1184,0.0


In [ ]:
datas.shape

(23111420, 8)

In [ ]:
# dateの最後一桁のみ残す
datas["date"] = datas["date"].apply(lambda x:x[2:])

In [ ]:
datas.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,date,sell_num
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1184,0.0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1184,0.0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1184,0.0


In [ ]:
datas["date"] = datas["date"].astype("int16")

In [ ]:
datas = reduce_mem_usage(datas,True)

Mem. usage decreased to 396.73 Mb (25.0% reduction)


In [ ]:
calender = pd.read_csv("calendar.csv",engine="python",encoding="utf-8")
calender.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
calender = encode_categorical(
   calender, calender.columns[7:11],
).pipe(reduce_mem_usage)

Mem. usage decreased to  0.07 Mb (61.4% reduction)


In [ ]:
calender

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,30,4,4,2,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,30,4,4,2,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,30,4,4,2,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,30,4,4,2,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,30,4,4,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,30,4,4,2,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,30,4,4,2,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,30,4,4,2,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,30,4,4,2,0,0,0


In [ ]:
calender.drop(["weekday", "year", "wday", "month"], axis=1, inplace=True)

In [ ]:
calender.head(2)

,date,wm_yr_wk,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,d_1,30,4,4,2,0,0,0
1,2011-01-30,11101,d_2,30,4,4,2,0,0,0


In [ ]:
calender["d"] = calender["d"].apply(lambda x:x[2:])
calender["d"] = calender["d"].astype("int16")
datas["d"] = datas["date"]
datas.drop("date",axis=1,inplace=True)

In [ ]:
# calenderとマージ
datas = pd.merge(datas,calender,how='left',on=["d"])

In [ ]:
datas.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,sell_num,d,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0.0,1184,2014-04-26,11413,30,4,4,2,0,0,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0.0,1184,2014-04-26,11413,30,4,4,2,0,0,0


In [ ]:
datas.shape

(23111420, 17)

In [ ]:
#priceも扱う
price = pd.read_csv("sell_prices.csv")
price = patair.reduce_mem_usage(price,True)
price.head(3)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812


In [ ]:
calender.drop(["d"],axis=1,inplace=True)

In [ ]:
# calenderとpriceをマージ
calender = pd.merge(calender,price)

In [ ]:
calender.head(2)

,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,store_id,item_id,sell_price
0,2011-01-29,11101,30,4,4,2,0,0,0,CA_1,HOBBIES_1_008,0.459961
1,2011-01-29,11101,30,4,4,2,0,0,0,CA_1,HOBBIES_1_009,1.559570


In [ ]:
calender.drop("wm_yr_wk", axis=1, inplace=True)

In [ ]:
calender.dtypes

date             object
event_name_1       int8
event_type_1       int8
event_name_2       int8
event_type_2       int8
snap_CA            int8
snap_TX            int8
snap_WI            int8
store_id         object
item_id          object
sell_price      float16
dtype: object

In [ ]:
calender["id"] = calender["item_id"] + "_" + calender["store_id"]

In [ ]:
calender.drop(["store_id","item_id","event_name_1", "event_name_2", "event_type_1", "event_type_2","snap_CA","snap_TX","snap_WI"], axis=1, inplace=True)

In [ ]:
calender["id"] = calender["id"].apply(lambda x:x + "_validation")

In [ ]:
datas.drop("d",axis=1,inplace=True)

In [ ]:
# 値段も含めてマージ
datas = pd.merge(datas, calender, on=['id','date'], how='left')

In [ ]:
datas.shape

(23111420, 17)

In [ ]:
# 未売期間のデータを削除
datas.dropna(subset=["sell_price"], inplace=True)

In [ ]:
datas.to_pickle("all_sales_train_validation.pickle")